In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

data_path="deu.txt"

In [2]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) -1)]:
    input_text, target_text, _ = line.split('\t')
    
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for input', max_encoder_seq_length)
print('Max sequence length of output', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 83
Max sequence length for input 15
Max sequence length of output 51


In [10]:
input_token_index = dict(
    [(char, i) for i ,char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i ,char in enumerate(target_characters)])

In [13]:
encoder_input_data = np.zeros(
            (len(input_texts),max_encoder_seq_length,num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros(
            (len(input_texts),max_decoder_seq_length,num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros(
            (len(input_texts),max_decoder_seq_length,num_decoder_tokens), dtype='float32')

In [14]:
for i, (input_text, target_text) in enumerate (zip(input_texts, target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t, input_token_index[char]]=1.
    encoder_input_data[i,t+1:,input_token_index[' ']]=1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] =1.
        if t>0:
            decoder_target_data[i,t - 1,target_token_index[char]]=1.
    
    decoder_input_data[i,t+1:,target_token_index[' ']] = 1.
    decoder_target_data[i,t:, target_token_index[' ']]=1.

In [15]:
encoder_inputs = Input(shape=(None, num_decoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c =encoder(encoder_inputs)

encoder_states = [state_h,state_c]

In [16]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [21]:
model = Model([encoder_inputs, decoder_inputs],decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorial_crossentropy', metrics=['accuracy'])


model.fit([encoder_input_data, decoder_input_data],decoder_target_data, 
          batch_size=batch_size,
          epochs=epochs,
         validation_split=0.3)

Epoch 1/100


ValueError: in user code:

    C:\Users\Gaurav\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Gaurav\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Gaurav\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Gaurav\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Gaurav\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Gaurav\anaconda3\envs\tensorflow\lib\site-pack

In [25]:
decoder_input_data

array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.